# Account Level Data

## Download Kaggle Datasets

data/raw/eth_illicit_features.csv <br>
`https://www.kaggle.com/datasets/gescobero/ethereum-fraud-dataset`

data/raw/transaction_dataset.csv <br>
`https://www.kaggle.com/datasets/vagifa/ethereum-frauddetection-dataset`

## Merge Kaggle Datasets

In [2]:
import pandas as pd

mapping = {
    'Address': 'address',
    'FLAG': 'flag',
    'Avg min between sent tnx': 'avgTimeBetweenSentTnx',
    'Avg min between received tnx': 'avgTimeBetweenRecTnx',
    'Time Diff between first and last (Mins)': 'lifetime',
    'Sent tnx': 'sentTransactions',
    'Received Tnx': 'receivedTransactions',
    'Number of Created Contracts': 'createdContracts',
    'Unique Received From Addresses': 'numUniqRecAddress',
    'Unique Sent To Addresses': 'numUniqSentAddress',
    'min value received': 'minValReceived',
    'max value received ': 'maxValReceived',
    'avg val received': 'avgValReceived',
    'min val sent': 'minValSent',
    'max val sent': 'maxValSent',
    'avg val sent': 'avgValSent',
    'total transactions (including tnx to create contract': 'totalTransactions',
    'total Ether sent': 'totalEtherSent',
    'total ether received': 'totalEtherReceived',
    'total ether sent contracts': 'totalEtherSentContracts',
    'total ether balance': 'totalEtherBalance',
}

df_kaggle_1 = pd.read_csv('data/raw/eth_illicit_features.csv') \
    [mapping.values()]
df_kaggle_2 = pd.read_csv('data/raw/transaction_dataset.csv') \
    .drop(columns=['Unnamed: 0', 'Index']) \
    .rename(columns=mapping) \
    [mapping.values()]

df_account = pd.concat([df_kaggle_1, df_kaggle_2]) \
    .drop_duplicates(subset='address') \
    .reset_index(drop=True) \
    .to_csv('data/processed/kaggle_transaction_dataset.csv', index=False)
    
df_account = pd.read_csv('data/processed/kaggle_transaction_dataset.csv')
df_account.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20302 entries, 0 to 20301
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   address                  20302 non-null  object 
 1   flag                     20302 non-null  int64  
 2   avgTimeBetweenSentTnx    20302 non-null  float64
 3   avgTimeBetweenRecTnx     20302 non-null  float64
 4   lifetime                 20302 non-null  float64
 5   sentTransactions         20302 non-null  int64  
 6   receivedTransactions     20302 non-null  int64  
 7   createdContracts         20302 non-null  int64  
 8   numUniqRecAddress        20302 non-null  int64  
 9   numUniqSentAddress       20302 non-null  int64  
 10  minValReceived           20302 non-null  float64
 11  maxValReceived           20302 non-null  float64
 12  avgValReceived           20302 non-null  float64
 13  minValSent               20302 non-null  float64
 14  maxValSent            

# Transaction Level Data

## Query Etherscan API

In [ ]:
import requests
import pandas as pd

def call(address, flag):
    # Define the API endpoint URL
    url = 'https://api.etherscan.io/api'

    # Define the parameters for the API request
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': address,
        'startblock': '0', 
        'endblock': '99999999', 
        'apikey': 'KCMY95HPGBY5CKC9PYY8FSVVAB89U5EZKU',  # Replace with your actual API key token
    }

    # Make the API request
    response = requests.get(url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        result = {
            'timeStamp': [], 
            'from': [], 
            'to': [], 
            'value': [], 
            'gasUsed': [], 
            'flag': [], 
        }

        # Extract the balance from the response
        for tx in data['result']:
            result['timeStamp'].append(tx['timeStamp'])
            result['from'].append(tx['from'])
            result['to'].append(tx['to'])
            result['value'].append(tx['value'])
            result['gasUsed'].append(tx['gasUsed'])
            result['flag'].append(flag)
        
        return result
    else:
        print(f'Error: {response.status_code} - {response.text}')

df_account = pd.read_csv('data/processed/kaggle_transaction.csv')
flag_mapping = df_account.set_index('address')['flag'].to_dict()

flag = []
frm = []
to = []
time = []
val = []
gas = []

for index, row in df_account.iterrows():
    result = call(row['address'], flag_mapping[row['address']])
    flag.extend(result['flag'])
    frm.extend(result['from'])
    to.extend(result['to'])
    time.extend(result['timeStamp'])
    val.extend(result['value'])
    gas.extend(result['gasUsed'])

    if index % 500 == 499:
        df_result = pd.DataFrame(data={
            'flag':flag, 
            'address from': frm, 
            'address to': to, 
            'timestamp':time, 
            'amount':val, 
            'gasUsed':gas, 
        })

        full_df.to_csv(f'transactions_{i // 500}.csv', index=False)
        flag = []
        frm = []
        to = []
        time = []
        val = []
        gas = []

## Merge EtherScan Datasets

In [ ]:
import pandas as pd

merged_df = pd.DataFrame()
i = 0
while True:
    try:
        df = pd.read_csv(f'data/queried/transactions_{i}.csv')
        merged_df = pd.concat([merged_df, df])
        i += 1
    except:
        full_df.to_csv('data/queried/full_transactions.csv', index=False)
        break